In [1]:
import torch
import json_repair
import pandas as pd

from tqdm import tqdm
from datasets import Dataset

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    #model_name="unsloth/Qwen2.5-7B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.47.0.
   \\   /|    GPU: NVIDIA RTX A6000. Max memory: 47.529 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"], 
    use_rslora=True,
    use_gradient_checkpointing=True
)

Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## data

In [4]:
df = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/dataset_files/comics_dataset_w_spanish.csv")

In [5]:
df

,file_name,page_nr,panel_nr,balloon_nr,utterance,Translated text in Spanish; Castilian,raw_annotation,raw_emotion,raw_speaker_id,emotion,speaker_id,split
0,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,2,1,DID YOU HAVE TO ELECTROCUTE HER SO HARD?,¿Tenías que electrocutarla tan fuerte?,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-1,AN0-DI0-FE3-SA0-SU5-JO0,ID-1,TEST
1,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,2,2,IT'S NOT LIKE I HAVE DIFFERENT SETTINGS.,Ni que tuviera diferentes niveles.,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-2,AN0-DI0-FE0-SA0-SU5-JO0,ID-2,TEST
2,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,2,3,YOU'RE ELECTROCUTIONER. IT'S YOUR WHOLE THING....,Eres Electrocutador. Es lo tuyo. Cabría pensar...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-1,AN0-DI0-FE2-SA0-SU0-JO0,ID-1,TEST
3,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,3,1,"OH, HEY. I THINK SHE'S AWAKE.","Eh, mira. Creo que está despierta.",2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-2,AN0-DI0-FE0-SA0-SU4-JO0,ID-2,TEST
4,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,4,1,"WELCOME BACK, MADAM MAYOR. BLOCKBUSTER IS PRET...","Bienvenida de nuevo, señora alcaldesa. Blockbu...",2024-08-27 - aselermekova20\nFeeling:AN3-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN3-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-1,AN3-DI0-FE0-SA0-SU0-JO0,ID-1,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...
7124,QC copy - 1737 - 34 The Walking Dead vol 15 - ...,21,3,2,SHE WOULDN'T DO THAT TO US. WE TALKED FOR A LO...,Ella no nos haría eso. Hemos hablado mucho tie...,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,\n2024-09-06 - SyimykRasulov\nSpokenBy:Eugene,AN0-DI0-FE1-SA3-SU0-JO0,Eugene,TRAIN
7125,QC copy - 1737 - 34 The Walking Dead vol 15 - ...,21,3,3,… I KNOW HER.,...La conozco.,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,\n2024-09-06 - SyimykRasulov\nSpokenBy:Eugene,AN0-DI0-FE1-SA3-SU0-JO0,Eugene,TRAIN
7126,QC copy - 1737 - 34 The Walking Dead vol 15 - ...,21,4,1,"UH, GUYS…","Hum, chicos...",2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,\n2024-09-06 - SyimykRasulov\nSpokenBy:JUANITA...,AN0-DI0-FE3-SA0-SU4-JO0,JUANITA SANCHEZ,TRAIN
7127,QC copy - 1737 - 34 The Walking Dead vol 15 - ...,22,1,1,PUT YOUR WEAPONS DOWN AND PUT YOUR HANDS IN TH...,Bajad las armas y levantad las manos... ¡ya!,2024-09-06 - SyimykRasulov\nFeeling:AN4-DI0-FE...,2024-09-06 - SyimykRasulov\nFeeling:AN4-DI0-FE...,\n2024-09-06 - SyimykRasulov\nSpokenBy:ID- 2,AN4-DI0-FE0-SA0-SU0-JO0,ID- 2,TRAIN


In [6]:
def get_emotions(row):

    utterance_emotions = row.emotion
    utterance_emotions_l = []
    emotion_class_labels = ["Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy"]

    if utterance_emotions == 'Neutral':
        
        utterance_emotions_l.append(utterance_emotions)
    
    else:
        utterance_emotions = utterance_emotions.split("-")

        for idx, emotion_annotation in enumerate(utterance_emotions):

            if '0' not in emotion_annotation:
                
                utterance_emotions_l.append(emotion_class_labels[idx])                

    return utterance_emotions_l

In [7]:
df['emotion_u'] = df.apply(lambda row: get_emotions(row), axis=1)

In [8]:
df

,file_name,page_nr,panel_nr,balloon_nr,utterance,Translated text in Spanish; Castilian,raw_annotation,raw_emotion,raw_speaker_id,emotion,speaker_id,split,emotion_u
0,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,2,1,DID YOU HAVE TO ELECTROCUTE HER SO HARD?,¿Tenías que electrocutarla tan fuerte?,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-1,AN0-DI0-FE3-SA0-SU5-JO0,ID-1,TEST,"[Fear, Surprise]"
1,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,2,2,IT'S NOT LIKE I HAVE DIFFERENT SETTINGS.,Ni que tuviera diferentes niveles.,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-2,AN0-DI0-FE0-SA0-SU5-JO0,ID-2,TEST,[Surprise]
2,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,2,3,YOU'RE ELECTROCUTIONER. IT'S YOUR WHOLE THING....,Eres Electrocutador. Es lo tuyo. Cabría pensar...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-1,AN0-DI0-FE2-SA0-SU0-JO0,ID-1,TEST,[Fear]
3,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,3,1,"OH, HEY. I THINK SHE'S AWAKE.","Eh, mira. Creo que está despierta.",2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN0-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-2,AN0-DI0-FE0-SA0-SU4-JO0,ID-2,TEST,[Surprise]
4,QC copy - 1500 - 04 Nightwing 19 _Nightwing 95...,1,4,1,"WELCOME BACK, MADAM MAYOR. BLOCKBUSTER IS PRET...","Bienvenida de nuevo, señora alcaldesa. Blockbu...",2024-08-27 - aselermekova20\nFeeling:AN3-DI0-F...,2024-08-27 - aselermekova20\nFeeling:AN3-DI0-F...,2024-09-05 - aidaraliev12345\nSpokenBy:ID-1,AN3-DI0-FE0-SA0-SU0-JO0,ID-1,TEST,[Anger]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7124,QC copy - 1737 - 34 The Walking Dead vol 15 - ...,21,3,2,SHE WOULDN'T DO THAT TO US. WE TALKED FOR A LO...,Ella no nos haría eso. Hemos hablado mucho tie...,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,\n2024-09-06 - SyimykRasulov\nSpokenBy:Eugene,AN0-DI0-FE1-SA3-SU0-JO0,Eugene,TRAIN,"[Fear, Sadness]"
7125,QC copy - 1737 - 34 The Walking Dead vol 15 - ...,21,3,3,… I KNOW HER.,...La conozco.,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,\n2024-09-06 - SyimykRasulov\nSpokenBy:Eugene,AN0-DI0-FE1-SA3-SU0-JO0,Eugene,TRAIN,"[Fear, Sadness]"
7126,QC copy - 1737 - 34 The Walking Dead vol 15 - ...,21,4,1,"UH, GUYS…","Hum, chicos...",2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,2024-09-06 - SyimykRasulov\nFeeling:AN0-DI0-FE...,\n2024-09-06 - SyimykRasulov\nSpokenBy:JUANITA...,AN0-DI0-FE3-SA0-SU4-JO0,JUANITA SANCHEZ,TRAIN,"[Fear, Surprise]"
7127,QC copy - 1737 - 34 The Walking Dead vol 15 - ...,22,1,1,PUT YOUR WEAPONS DOWN AND PUT YOUR HANDS IN TH...,Bajad las armas y levantad las manos... ¡ya!,2024-09-06 - SyimykRasulov\nFeeling:AN4-DI0-FE...,2024-09-06 - SyimykRasulov\nFeeling:AN4-DI0-FE...,\n2024-09-06 - SyimykRasulov\nSpokenBy:ID- 2,AN4-DI0-FE0-SA0-SU0-JO0,ID- 2,TRAIN,[Anger]


In [9]:
#df_train = df[df.split == "TRAIN"].reset_index(drop=True)

In [10]:
#len(df_train)

### Comics dataset

In [11]:
# def build_instruction():
   
#     emotion_classes = ["anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"]
#     formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])
    
#     instruction = f"""### Emotion Analysis Expert Role

# You are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Your task is to analyze utterances and identify their emotional content.

# INPUT:
# - You will receive a list of utterances from a page in a comic book
# - The utterance may express one or multiple emotions
# - The name and summary of the comic book

# TASK:
# 1. Carefully analyze the emotional context and tone of each utterance in the page
# 2. Identify applicable emotions from the following classes:
#    {formatted_classes}
# 3. For each utterance in a comic page, identify all emotions present and return an array of emotion arrays in order.

# RULES:
# 1. Use ONLY the labels listed above
# 2. Output must be a JSON with single key "page_utterance_emotions"
# 3. Value must be an array where:
#    - Each element is an array of emotions for one utterance
#    - Order matches the input utterances order
#    - Multiple emotions are allowed per utterance
# 4. No explanations, only JSON output

# IMPORTANT:
# - Each array element corresponds to one utterance
# - One utterance can have multiple emotions
# - Maintain exact spelling and case of emotion labels
# - Keep emotions in arrays even for single emotions

# """
#     return instruction

In [12]:
def build_instruction():
    """
    Builds an instruction string for a bilingual emotion analysis expert system
    that processes both English and Spanish comic book dialogue and returns
    structured JSON output.
    
    Returns:
        str: A formatted instruction string containing the role description,
             output format requirements, and available emotion classes.
    """
    emotion_classes = [
        "anger",
        "disgust", 
        "fear",
        "sadness",
        "surprise",
        "joy",
        "neutral"
    ]
    
    formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])
    
    instruction = f"""### Bilingual Emotion Analysis Expert Role

You are an advanced emotion analysis expert specializing in comic book dialogue interpretation
in both English and Spanish. Your task is to analyze bilingual utterances and identify
their emotional content.

For each utterance, you will receive:
- The English version
- The Spanish version (traducción al español)

You must return your analysis as a JSON object with exactly this structure:
{{
    "emotions": <emotion_class>
}}

Where <emotion_class> must be one or more of: {formatted_classes}

Guidelines:
- Consider both language versions when determining the emotion
- Analyze contextual and linguistic cues in both languages
- Return only the specified JSON structure with no additional explanation

OUTPUT REQUIREMENTS:
- Format: JSON object with a single key "emotions"
- Value: Array of one or more emotion classes as strings

Example input:
English: "I can't believe you did this to me!"
Spanish: "¡No puedo creer que me hayas hecho esto!"

Example output:
{{"emotions": ["anger", "surpise"]}}
"""

    return instruction

In [13]:
# def build_instruction():
  
#     emotion_classes = ["Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy", "Neutral"]
#     formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])
    
#     instruction = f"""### Emotion Analysis Expert Role

# You are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Your task is to analyze utterances and identify their emotional content while considering conversational context.

# INPUT:
# - You will receive a list of 6 consecutive utterances from a comic book:
#   * The first 5 utterances provide conversational context
#   * The 6th (last) utterance is the one to be classified
# - Each utterance may express one or multiple emotions

# TASK:
# 1. Read through the context utterances to understand the emotional flow
# 2. Carefully analyze the emotional context, tone, and potential emotional shifts in the final utterance
# 3. Identify applicable emotions for the final utterance only from the following classes:
#    {formatted_classes}

# OUTPUT REQUIREMENTS:
# - Format: JSON object with a single key "list_emotion_classes"
# - Value: Array of one or more emotion classes as strings
# - The classification should be for the final utterance only, using previous utterances as context

# IMPORTANT NOTES:
# - Do not include any explanations in the output, only the JSON object
# - Use the context to better understand emotional transitions and current emotional state
# - Context utterances may reveal emotional buildup or shifts that influence the final utterance

# """
#     return instruction

In [14]:
instruction = build_instruction()

In [15]:
sys_msg_l = []
user_msg_l = []
assistant_msg_l = []

for _, row in df.iterrows():
        
        sys_msg = {'role': 'system', 'content': instruction}
        
        utterance_en = row['utterance']
        utterance_es = row['Translated text in Spanish; Castilian']
        
        # prompt = "Here is the utterance in English and Spanish: "
        
        
        # utterances_l = eval(row['utterance'])
        # pg_utterances = "\n".join(f"{i+1}. {title}" for i, title in enumerate(utterances_l))
        
        usr_prompt = f"""English: {utterance_en}
        Spanish: {utterance_es}"""
        
        user_msg = {'role': 'user', 'content': usr_prompt}

        #obj = {"list_emotion_classes": row['emotion_u']}
        #obj = row['emotion_u']
        #assistant_msg = {'from': 'gpt', 'value': obj}
        #assistant_msg = {'role': 'assistant', 'content': f'{{"list_emotion_classes": {row["emotion_c"]}}}'}
        
        assistant_msg = {'role': 'assistant', 'content': ""}


        sys_msg_l.append(sys_msg)
        user_msg_l.append(user_msg)
        assistant_msg_l.append(assistant_msg)
        

In [16]:
comics_dataset = []

for i in range(len(sys_msg_l)):

    comics_dataset.append([sys_msg_l[i], user_msg_l[i], assistant_msg_l[i]])

In [17]:
len(comics_dataset)

7129

In [18]:
comics_dataset[0]

[{'role': 'system',
  'content': '### Bilingual Emotion Analysis Expert Role\n\nYou are an advanced emotion analysis expert specializing in comic book dialogue interpretation\nin both English and Spanish. Your task is to analyze bilingual utterances and identify\ntheir emotional content.\n\nFor each utterance, you will receive:\n- The English version\n- The Spanish version (traducción al español)\n\nYou must return your analysis as a JSON object with exactly this structure:\n{\n    "emotions": <emotion_class>\n}\n\nWhere <emotion_class> must be one or more of: "anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"\n\nGuidelines:\n- Consider both language versions when determining the emotion\n- Analyze contextual and linguistic cues in both languages\n- Return only the specified JSON structure with no additional explanation\n\nOUTPUT REQUIREMENTS:\n- Format: JSON object with a single key "emotions"\n- Value: Array of one or more emotion classes as strings\n\nExample input:

In [19]:
model = FastLanguageModel.for_inference(model)

In [20]:
inputs = tokenizer.apply_chat_template(
            comics_dataset,
            padding=True,
            truncation=True,
            add_generation_prompt=True,
            return_dict=True,
            return_tensors="pt",
)

In [21]:
def batch_tensor(tensor, batch_size):
    return [tensor[i:i+batch_size] for i in range(0, tensor.size(0), batch_size)]

In [22]:
BATCH_SIZE = 128
input_ids_batches = batch_tensor(inputs['input_ids'], BATCH_SIZE) # type: ignore
attention_mask_batches = batch_tensor(inputs['attention_mask'], BATCH_SIZE) # type: ignore

In [23]:
generated_outputs = []

In [24]:
for i, (input_ids_batch, attention_mask_batch) in tqdm(enumerate(zip(input_ids_batches, attention_mask_batches))):
    
    print(f"Processing batch {i + 1}")
    
    # Move tensors to model device
    inputs = {
        'input_ids': input_ids_batch.to(model.device), # type: ignore
        'attention_mask': attention_mask_batch.to(model.device) # type: ignore
    }
    
    generated = model.generate(**inputs, max_new_tokens=32, pad_token_id=tokenizer.eos_token_id, do_sample=True,
     temperature=0.1,
     top_p=0.9,)

    generated_outputs.append(generated)

0it [00:00, ?it/s]

Processing batch 1


1it [00:23, 23.97s/it]

Processing batch 2


2it [00:47, 23.68s/it]

...

54it [19:55, 23.03s/it]

Processing batch 55


55it [20:18, 23.27s/it]

Processing batch 56


56it [20:36, 22.07s/it]


In [25]:
decoded_outputs = []

for batch in generated_outputs:

    for prediction in batch:

        decoded_outputs.append(tokenizer.decode(prediction, skip_special_tokens=True))

In [26]:
len(decoded_outputs)

7129

In [27]:
decoded_outputs

['system\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n### Bilingual Emotion Analysis Expert Role\n\nYou are an advanced emotion analysis expert specializing in comic book dialogue interpretation\nin both English and Spanish. Your task is to analyze bilingual utterances and identify\ntheir emotional content.\n\nFor each utterance, you will receive:\n- The English version\n- The Spanish version (traducción al español)\n\nYou must return your analysis as a JSON object with exactly this structure:\n{\n    "emotions": <emotion_class>\n}\n\nWhere <emotion_class> must be one or more of: "anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"\n\nGuidelines:\n- Consider both language versions when determining the emotion\n- Analyze contextual and linguistic cues in both languages\n- Return only the specified JSON structure with no additional explanation\n\nOUTPUT REQUIREMENTS:\n- Format: JSON object with a single key "emotions"\n- Value: Array of one or more 

In [32]:
raw_preds = []

for op in decoded_outputs:
    raw_preds.append(op.split("assistant\n\nassistant\n\n")[1])
    

In [33]:
len(raw_preds)

7129

In [34]:
raw_preds

['{\n    "emotions": ["anger", "surprise"]\n}',
 '{\n    "emotions": ["neutral"]\n}',
 '{\n    "emotions": ["anger", "disgust", "surprise"]\n}',
 '{\n    "emotions": ["neutral"]\n}',
 '{\n    "emotions": ["anger", "neutral"]\n}',
 '{\n    "emotions": ["neutral"]\n}',
 '{\n    "emotions": ["anger"]\n}',
 '{\n    "emotions": ["neutral", "neutral"]\n}',
 '{\n    "emotions": ["neutral"]\n}',
 '{\n    "emotions": ["anger", "surprise"]\n}',
 '{\n    "emotions": ["neutral"]\n}',
 '{\n  "emotions": ["neutral"]\n}',
 '{\n    "emotions": ["neutral"]\n}',
 '{\n    "emotions": ["anger", "sadness"]\n}',
 '{\n    "emotions": ["neutral"]\n}',
 '{\n    "emotions": ["anger", "disgust", "fear", "sadness"]\n}',
 '{\n    "emotions": ["neutral"]\n}',
 '{\n    "emotions": ["disappointment", "sadness"]\n}',
 '{\n    "emotions": ["neutral"]\n}',
 '{\n    "emotions": ["anger", "fear"]\n}',
 '{\n    "emotions": ["anger", "fear"]\n}',
 '{\n    "emotions": ["anger"]\n}',
 '{\n    "emotions": ["surprise"]\n}',
 '{

In [35]:
predictions = [json_repair.loads(e) for e in raw_preds]

In [36]:
len(predictions)

7129

In [37]:
predictions

[{'emotions': ['anger', 'surprise']},
 {'emotions': ['neutral']},
 {'emotions': ['anger', 'disgust', 'surprise']},
 {'emotions': ['neutral']},
 {'emotions': ['anger', 'neutral']},
 {'emotions': ['neutral']},
 {'emotions': ['anger']},
 {'emotions': ['neutral', 'neutral']},
 {'emotions': ['neutral']},
 {'emotions': ['anger', 'surprise']},
 {'emotions': ['neutral']},
 {'emotions': ['neutral']},
 {'emotions': ['neutral']},
 {'emotions': ['anger', 'sadness']},
 {'emotions': ['neutral']},
 {'emotions': ['anger', 'disgust', 'fear', 'sadness']},
 {'emotions': ['neutral']},
 {'emotions': ['disappointment', 'sadness']},
 {'emotions': ['neutral']},
 {'emotions': ['anger', 'fear']},
 {'emotions': ['anger', 'fear']},
 {'emotions': ['anger']},
 {'emotions': ['surprise']},
 {'emotions': ['surprise']},
 {'emotions': ['neutral']},
 {'emotions': ['joy']},
 {'emotions': ['anger', 'surprise']},
 {'emotions': ['neutral']},
 {'emotions': ['anger', 'joy']},
 {'emotions': ['anger', 'defiance']},
 {'emotions':

In [38]:
grounds = df.emotion_u.tolist()

In [39]:
len(grounds)

7129

In [40]:
grounds

[['Fear', 'Surprise'],
 ['Surprise'],
 ['Fear'],
 ['Surprise'],
 ['Anger'],
 ['Anger', 'Surprise'],
 ['Joy'],
 ['Joy'],
 ['Fear'],
 ['Surprise'],
 ['Joy'],
 ['Fear', 'Surprise'],
 ['Joy'],
 ['Fear'],
 ['Joy'],
 ['Fear', 'Sadness'],
 ['Surprise'],
 ['Anger', 'Sadness'],
 ['Anger'],
 ['Anger'],
 ['Anger'],
 ['Fear'],
 ['Surprise'],
 ['Surprise'],
 ['Fear', 'Surprise'],
 ['Fear'],
 ['Anger'],
 ['Surprise'],
 ['Anger', 'Joy'],
 ['Anger'],
 ['Joy'],
 ['Surprise'],
 ['Joy'],
 ['Surprise'],
 ['Fear'],
 ['Surprise'],
 ['Joy'],
 ['Fear'],
 ['Fear', 'Joy'],
 ['Fear', 'Surprise'],
 ['Joy'],
 ['Sadness', 'Surprise'],
 ['Surprise'],
 ['Anger', 'Sadness'],
 ['Anger', 'Sadness'],
 ['Anger'],
 ['Anger'],
 ['Anger', 'Disgust'],
 ['Neutral'],
 ['Surprise'],
 ['Fear'],
 ['Surprise'],
 ['Fear', 'Joy'],
 ['Fear', 'Joy'],
 ['Surprise'],
 ['Fear', 'Joy'],
 ['Fear', 'Joy'],
 ['Joy'],
 ['Fear', 'Sadness'],
 ['Surprise'],
 ['Sadness'],
 ['Fear', 'Surprise'],
 ['Fear'],
 ['Surprise'],
 ['Sadness'],
 ['Joy'],
 ['

In [41]:
preds_l = []
bad_idx = []

for i, pred in enumerate(predictions):
    try:        
        preds_l.append(pred['emotions'])
    except:
        print(i)
        bad_idx.append(i)

44
389
393
456
457
504
597
651
1174
1179
1226
1232
1244
1414
1643
1728
1759
2615
2659
2898
3369
3986
4060
4149
4216
4249
4433
4840
5088
5890
6015
6135
6423
6739
6762
6845
6847
6865
6891
6900
6985
7067


In [62]:
predictions[393]

''

In [43]:
len(bad_idx)

42

In [44]:
grounds = [item for i, item in enumerate(grounds) if i not in bad_idx]

In [45]:
len(grounds), len(preds_l)

(7087, 7087)

In [46]:
bad_idx = []

for idx, (i,j) in enumerate(zip(grounds, preds_l)):
    if len(i) != len(j):
        print(idx, len(i), len(j))
        bad_idx.append(idx)

2 1 3
4 1 2
5 2 1
7 1 2
9 1 2
11 2 1
13 1 2
15 2 4
19 1 2
20 1 2
24 2 1
26 1 2
29 1 2
34 1 2
35 1 2
37 1 2
40 1 2
42 1 2
50 1 2
51 2 3
54 2 1
57 2 3
58 1 2
60 2 1
61 1 3
62 1 2
63 1 2
64 1 2
65 1 2
67 1 3
69 1 3
72 1 2
73 1 2
74 1 2
75 1 2
77 1 2
78 1 2
80 1 2
82 1 2
83 1 2
85 1 2
86 1 3
88 1 2
90 1 2
91 1 2
92 1 2
98 1 2
100 2 1
101 1 2
103 2 1
104 1 2
106 1 2
107 1 2
108 1 3
110 1 2
111 1 2
112 1 2
113 1 2
114 2 1
118 3 2
120 1 2
121 1 2
122 1 2
124 1 2
126 2 1
127 2 1
128 1 2
129 1 2
130 1 2
133 2 3
138 1 2
142 2 1
143 2 1
144 2 1
145 2 1
147 2 1
148 2 1
149 1 2
150 2 1
152 2 1
153 2 1
154 1 2
156 2 1
158 1 2
160 1 2
164 2 1
165 1 2
166 1 2
169 1 2
170 1 2
172 2 1
173 1 2
174 1 2
177 1 2
179 2 1
180 1 2
184 2 1
191 2 1
195 2 3
197 1 2
198 1 2
201 1 2
203 1 2
205 1 2
206 2 1
207 1 2
212 2 1
214 1 2
215 2 1
217 2 1
219 2 1
220 1 2
221 2 1
223 1 2
224 2 1
226 2 1
230 2 1
231 3 2
233 2 1
235 1 2
236 1 2
240 2 1
241 2 1
243 1 2
244 1 2
245 1 2
247 1 2
248 1 2
251 2 1
254 1 2
256 1 2
257 

In [47]:
bad_idx.sort(reverse=True)

# Remove elements from 'grounds' at the specified indices
for idx in bad_idx:
    
    del grounds[idx]
    del preds_l[idx]

In [48]:
len(grounds), len(preds_l)

(3490, 3490)

In [55]:
grounds

[['Fear', 'Surprise'],
 ['Surprise'],
 ['Surprise'],
 ['Joy'],
 ['Fear'],
 ['Joy'],
 ['Joy'],
 ['Joy'],
 ['Surprise'],
 ['Anger', 'Sadness'],
 ['Anger'],
 ['Fear'],
 ['Surprise'],
 ['Surprise'],
 ['Fear'],
 ['Surprise'],
 ['Anger', 'Joy'],
 ['Joy'],
 ['Surprise'],
 ['Joy'],
 ['Surprise'],
 ['Joy'],
 ['Fear', 'Joy'],
 ['Fear', 'Surprise'],
 ['Sadness', 'Surprise'],
 ['Anger', 'Sadness'],
 ['Anger'],
 ['Anger'],
 ['Anger', 'Disgust'],
 ['Neutral'],
 ['Surprise'],
 ['Fear'],
 ['Fear', 'Joy'],
 ['Surprise'],
 ['Fear', 'Joy'],
 ['Joy'],
 ['Sadness'],
 ['Surprise', 'Joy'],
 ['Fear'],
 ['Surprise'],
 ['Joy'],
 ['Fear'],
 ['Fear'],
 ['Fear'],
 ['Fear'],
 ['Anger'],
 ['Fear', 'Surprise'],
 ['Fear', 'Surprise'],
 ['Fear', 'Surprise'],
 ['Fear', 'Surprise'],
 ['Fear', 'Surprise'],
 ['Anger'],
 ['Anger'],
 ['Anger'],
 ['Anger'],
 ['Sadness'],
 ['Surprise'],
 ['Surprise'],
 ['Fear', 'Surprise'],
 ['Fear', 'Sadness'],
 ['Surprise'],
 ['Anger'],
 ['Anger'],
 ['Fear'],
 ['Anger'],
 ['Anger'],
 ['Sadne

In [56]:
grounds = [[item.lower() for item in sublist] for sublist in grounds]

In [57]:
mlb = MultiLabelBinarizer()

In [58]:
y_true_mhot = mlb.fit_transform(grounds)
y_pred_mhot = mlb.transform(preds_l)

/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['admiration', 'affection', 'aggression', 'amusement', 'anticipation', 'apology', 'assertion', 'assertive', 'assertiveness', 'authority', 'awe', 'blame', 'care', 'caution', 'command', 'compassion', 'concern', 'confidence', 'confusion', 'contempt', 'curiosity', 'defensiveness', 'defiance', 'denial', 'desire', 'desperation', 'determination', 'devotion', 'disappointment', 'disbelief', 'disrespect', 'embarrassment', 'empathy', 'encouragement', 'envy', 'excitement', 'faith', 'formality', 'frustration', 'gratitude', 'guilt', 'hate', 'hatred', 'hope', 'hunger', 'indignation', 'insult', 'irony', 'love', 'neutrality', 'nostalgia', 'playfulness', 'protectiveness', 'regret', 'relief', 'resolute', 'respect', 'revenge', 'sarcasm', 'sarcastic', 'sarcastic_joy', 'shock', 'sigh', 'skepticism', 'suspicion', 'sympathy', 'threat', 'trust', 'urgency'] will be ignor

In [59]:
y_true_mhot.shape, y_pred_mhot.shape

((3490, 7), (3490, 7))

In [60]:
print(classification_report(y_true_mhot, y_pred_mhot, target_names=mlb.classes_, digits=3))

              precision    recall  f1-score   support

       anger      0.631     0.454     0.528      1093
     disgust      0.254     0.304     0.277       161
        fear      0.570     0.256     0.353       734
         joy      0.604     0.363     0.453       910
     neutral      0.134     0.907     0.233       291
     sadness      0.709     0.198     0.309       800
    surprise      0.625     0.324     0.426       887

   micro avg      0.393     0.363     0.377      4876
   macro avg      0.504     0.401     0.369      4876
weighted avg      0.586     0.363     0.407      4876
 samples avg      0.367     0.344     0.352      4876



/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [164]:
# raw_outputs = []

# for message in tqdm(comics_dataset):
    
#     inputs = tokenizer.apply_chat_template(
#     message,
#     tokenize=True,
#     add_generation_prompt=True,
#     #return_dict=True,
#     return_tensors="pt",
# ).to("cuda")
#     #print(inputs)
#     #break
    
#     output = model.generate(input_ids=inputs, max_new_tokens=128)[0]
#     #output = model.generate(**inputs, max_new_tokens=128)[0]
    
#     input_length = inputs.shape[1]
#     generated_tokens = output[input_length:]
    
#     decoded_output = tokenizer.decode(generated_tokens, skip_special_tokens=True)  
#     #decoded_output = tokenizer.decode(output, skip_special_tokens=True)
#     raw_outputs.append(decoded_output)
#     #break

100%|██████████| 874/874 [56:19<00:00,  3.87s/it]  


In [165]:
raw_outputs

['{\n    "page_utterance_emotions": [\n        [\n            "anger"\n        ],\n        [\n            "anger",\n            "disgust"\n        ],\n        [\n            "fear"\n        ],\n        [\n            "fear"\n        ],\n        [\n            "anger",\n            "disgust"\n        ],\n        [\n            "sadness"\n        ],\n        [\n            "fear",\n            "surprise"\n        ],\n        [\n            "surprise"\n        ],\n        [\n            "surprise"\n        ],\n        [\n            "fear"\n        ]\n    ]\n}',
 '{\n  "page_utterance_emotions": [\n    ["anger", "disgust"],\n    ["surprise", "joy"],\n    ["anger", "fear"],\n    ["anger", "fear", "surprise"],\n    ["surprise", "joy"],\n    ["disgust"],\n    ["neutral"],\n    ["anger"],\n    ["anger", "joy"]\n  ]\n}',
 '{\n  "page_utterance_emotions": [\n    ["anger"],\n    ["anger", "surprise"],\n    ["anger", "disgust"],\n    ["joy", "anger"]\n  ]\n}',
 '```json\n{\n  "page_utterance_emot

In [39]:
# def fix_comics_dataset(comics_dataset):
#     fixed_comics_dataset = []
#     for conversation in comics_dataset:
#         fixed_conversation = []
#         for message in conversation:
#             if isinstance(message['content'], list):  # If the 'value' is a list of emotions
#                 message['content'] = ', '.join(message['content'])  # Join the list into a string
#             fixed_conversation.append(message)
#         fixed_comics_dataset.append(fixed_conversation)
#     return fixed_comics_dataset

In [40]:
#fixed_comics_dataset = fix_comics_dataset(comics_dataset)

In [41]:
# dataset = Dataset.from_dict({
#     'conversations': fixed_comics_dataset
# })

In [42]:
#dataset

In [43]:
#dataset[0]['conversations']

In [44]:
# tokenizer = get_chat_template(
#     tokenizer,
#     mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
#     chat_template="chatml",
# )

# def apply_template_comics(examples):
#     messages = examples["conversations"]
#     #messages = examples['input'] + examples['output']
#     text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in messages]
#     return {"text": text}

In [45]:
#comics_dataset = dataset.map(apply_template_comics, batched=True)

In [46]:
#comics_dataset

In [47]:
# def split_dataset(dataset, train_ratio=0.8):
#     train_test = dataset.train_test_split(test_size=1 - train_ratio)
#     return train_test

# dataset_split = split_dataset(comics_dataset)

In [48]:
# train_dataset = dataset_split['train']
# eval_dataset = dataset_split['test']

In [49]:
# train_dataset

In [50]:
# print(train_dataset[452]['text'])

In [51]:
# OUTPUT_DIR = "/Utilisateurs/umushtaq/emotion_analysis_comics/outputs_dir_tmp"

In [52]:
# args=TrainingArguments(
#         learning_rate=3e-4,
#         lr_scheduler_type="cosine",
#         per_device_train_batch_size=16,
#         gradient_accumulation_steps=2,
#         num_train_epochs=10,
#         fp16=not is_bfloat16_supported(),
#         bf16=is_bfloat16_supported(),
#         logging_steps=25,
#         optim="adamw_8bit",
#         weight_decay=0.01,
#         warmup_steps=10,
        
#         eval_strategy="steps",  # Run evaluation during training (can also use "epoch")
#         eval_steps=25,  # Perform evaluation every 50 steps
#         save_strategy="steps",  # Save the model every few steps
#         save_steps=25,  # Save every 200 steps
#         load_best_model_at_end=True,
    
#         output_dir=OUTPUT_DIR,
#         seed=0,
#     )

In [53]:
# trainer=SFTTrainer(
#     model=model,
#     tokenizer=tokenizer,
#     train_dataset=train_dataset,  # Replace with your train dataset
#     eval_dataset=eval_dataset, 
#     dataset_text_field="text",
#     max_seq_length=max_seq_length,
#     dataset_num_proc=2,
#     packing=False,
#     args=args,
# )

In [54]:
# trainer.train()

In [55]:
#model = FastLanguageModel.for_inference(model)

In [56]:
#df_test = df[df.split == "TEST"].reset_index(drop=True)

In [57]:
# sys_msg_l = []
# user_msg_l = []
# assistant_msg_l = []

# for _, row in df_test.iterrows():
        
#         sys_msg = {'role': 'system', 'content': instruction}
        
        
#         comics_title = row['comics_title']
#         comics_summary = row['comics_summary']
        
#         utterances_l = eval(row['utterance'])
#         pg_utterances = "\n".join(f"{i+1}. {title}" for i, title in enumerate(utterances_l))
        
#         usr_prompt = f"Comics title: {comics_title}\n" + f"Comics summary: {comics_summary}\n" + "Here is the list of utterances that you will classify: \n" + pg_utterances
        
#         user_msg = {'role': 'user', 'content': usr_prompt}

#         #obj = {"list_emotion_classes": row['emotion_u']}
#         #obj = row['emotion_u']
#         #assistant_msg = {'from': 'gpt', 'value': obj}
#         assistant_msg = {'role': 'assistant', 'content': ""}


#         sys_msg_l.append(sys_msg)
#         user_msg_l.append(user_msg)
#         assistant_msg_l.append(assistant_msg)
        

In [58]:
# test_messages = []

# for i in range(len(sys_msg_l)):
    
#     #obj = {"list_emotion_classes": ["Anger", "Fear"]}

#     #comics_dataset.append([human_msg_l[i], assistant_msg_l[i]])
#     test_messages.append([sys_msg_l[i], user_msg_l[i], assistant_msg_l[i]])

In [59]:
# human_msg_l = []
# assistant_msg_l = []

# for _, row in df_test.iterrows():
        
#         prompt = instruction.replace("<comic_title>", row['comics_title']).replace("<speaker_id>", row['speaker_id']).replace("<utterance>", row['utterance'])
        
#         human_msg = {'role': 'user', 'content': prompt}
        
#         #obj = {"list_emotion_classes": row['emotion_u']}
#         obj = row['emotion_u']
#         assistant_msg = {'role': 'assistant', 'content': ""}
        
#         human_msg_l.append(human_msg)
#         assistant_msg_l.append(assistant_msg)
        
        

In [60]:
# test_messages = []

# for i in range(len(human_msg_l)):
    
#     #obj = {"list_emotion_classes": ["Anger", "Fear"]}

#     test_messages.append([human_msg_l[i], assistant_msg_l[i]])

In [61]:
#len(test_messages)

In [62]:
# test_messages = test_messages[:100]

In [63]:
# raw_outputs = []

# for message in tqdm(test_messages):
    
#     inputs = tokenizer.apply_chat_template(
#     message,
#     tokenize=True,
#     add_generation_prompt=True,
#     #return_dict=True,
#     return_tensors="pt",
# ).to("cuda")
#     #print(inputs)
#     #break
    
#     output = model.generate(input_ids=inputs, max_new_tokens=128)[0]
#     #output = model.generate(**inputs, max_new_tokens=128)[0]
    
#     input_length = inputs.shape[1]
#     generated_tokens = output[input_length:]
    
#     decoded_output = tokenizer.decode(generated_tokens, skip_special_tokens=True)  
#     #decoded_output = tokenizer.decode(output, skip_special_tokens=True)
#     raw_outputs.append(decoded_output)
#     #break

In [64]:
#print(tokenizer.decode(inputs[0]))

In [166]:
len(raw_outputs)

874

In [167]:
raw_outputs

['{\n    "page_utterance_emotions": [\n        [\n            "anger"\n        ],\n        [\n            "anger",\n            "disgust"\n        ],\n        [\n            "fear"\n        ],\n        [\n            "fear"\n        ],\n        [\n            "anger",\n            "disgust"\n        ],\n        [\n            "sadness"\n        ],\n        [\n            "fear",\n            "surprise"\n        ],\n        [\n            "surprise"\n        ],\n        [\n            "surprise"\n        ],\n        [\n            "fear"\n        ]\n    ]\n}',
 '{\n  "page_utterance_emotions": [\n    ["anger", "disgust"],\n    ["surprise", "joy"],\n    ["anger", "fear"],\n    ["anger", "fear", "surprise"],\n    ["surprise", "joy"],\n    ["disgust"],\n    ["neutral"],\n    ["anger"],\n    ["anger", "joy"]\n  ]\n}',
 '{\n  "page_utterance_emotions": [\n    ["anger"],\n    ["anger", "surprise"],\n    ["anger", "disgust"],\n    ["joy", "anger"]\n  ]\n}',
 '```json\n{\n  "page_utterance_emot

In [168]:
grounds = df.emotion_c.tolist()

In [169]:
len(grounds)

874

In [170]:
import json

In [171]:
predictions = [json_repair.loads(e) for e in raw_outputs]

In [172]:
predictions

[{'page_utterance_emotions': [['anger'],
   ['anger', 'disgust'],
   ['fear'],
   ['fear'],
   ['anger', 'disgust'],
   ['sadness'],
   ['fear', 'surprise'],
   ['surprise'],
   ['surprise'],
   ['fear']]},
 {'page_utterance_emotions': [['anger', 'disgust'],
   ['surprise', 'joy'],
   ['anger', 'fear'],
   ['anger', 'fear', 'surprise'],
   ['surprise', 'joy'],
   ['disgust'],
   ['neutral'],
   ['anger'],
   ['anger', 'joy']]},
 {'page_utterance_emotions': [['anger'],
   ['anger', 'surprise'],
   ['anger', 'disgust'],
   ['joy', 'anger']]},
 {'page_utterance_emotions': [['anger']]},
 {'page_utterance_emotions': [['joy'],
   ['anger', 'joy'],
   ['anger'],
   ['anger', 'fear'],
   ['fear', 'surprise'],
   ['joy']]},
 {'page_utterance_emotions': [['anger'],
   ['fear'],
   ['anger'],
   ['fear'],
   ['fear', 'surprise'],
   ['fear'],
   ['fear', 'anger'],
   ['fear', 'anger'],
   ['joy'],
   ['joy', 'anger']]},
 {'page_utterance_emotions': [['joy'],
   ['anger', 'disgust'],
   ['surprise

In [173]:
# #predictions = [e.split('\n\n')[0] for e in raw_outputs]
# bad_idx = []
# predictions = []

# for idx, e in enumerate(raw_outputs):
#     try:
#         predictions.append(json.loads(e))
#     except:
#         print(idx)
#         bad_idx.append(idx)
        


In [174]:
#len(predictions)

In [175]:
#predictions

In [176]:
#predictions = [json_repair.loads(e) for e in predictions]

In [177]:
#predictions

In [178]:
preds_l = []
bad_idx = []

for i, pred in enumerate(predictions):
    try:        
        preds_l.append(pred['page_utterance_emotions'])
    except:
        print(i)
        bad_idx.append(i)

In [179]:
preds_l

[[['anger'],
  ['anger', 'disgust'],
  ['fear'],
  ['fear'],
  ['anger', 'disgust'],
  ['sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['surprise'],
  ['fear']],
 [['anger', 'disgust'],
  ['surprise', 'joy'],
  ['anger', 'fear'],
  ['anger', 'fear', 'surprise'],
  ['surprise', 'joy'],
  ['disgust'],
  ['neutral'],
  ['anger'],
  ['anger', 'joy']],
 [['anger'], ['anger', 'surprise'], ['anger', 'disgust'], ['joy', 'anger']],
 [['anger']],
 [['joy'],
  ['anger', 'joy'],
  ['anger'],
  ['anger', 'fear'],
  ['fear', 'surprise'],
  ['joy']],
 [['anger'],
  ['fear'],
  ['anger'],
  ['fear'],
  ['fear', 'surprise'],
  ['fear'],
  ['fear', 'anger'],
  ['fear', 'anger'],
  ['joy'],
  ['joy', 'anger']],
 [['joy'], ['anger', 'disgust'], ['surprise'], ['surprise', 'joy'], ['joy']],
 [['anger', 'fear'],
  ['fear', 'surprise'],
  ['anger', 'confusion'],
  ['joy', 'determination'],
  ['anger', 'defiance'],
  ['fear', 'uncertainty'],
  ['fear', 'caution'],
  ['anger', 'frustration'],
  ['anger',

In [180]:
grounds = [item for i, item in enumerate(grounds) if i not in bad_idx]

In [181]:
len(grounds), len(preds_l)

(874, 874)

In [182]:
preds_l

[[['anger'],
  ['anger', 'disgust'],
  ['fear'],
  ['fear'],
  ['anger', 'disgust'],
  ['sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['surprise'],
  ['fear']],
 [['anger', 'disgust'],
  ['surprise', 'joy'],
  ['anger', 'fear'],
  ['anger', 'fear', 'surprise'],
  ['surprise', 'joy'],
  ['disgust'],
  ['neutral'],
  ['anger'],
  ['anger', 'joy']],
 [['anger'], ['anger', 'surprise'], ['anger', 'disgust'], ['joy', 'anger']],
 [['anger']],
 [['joy'],
  ['anger', 'joy'],
  ['anger'],
  ['anger', 'fear'],
  ['fear', 'surprise'],
  ['joy']],
 [['anger'],
  ['fear'],
  ['anger'],
  ['fear'],
  ['fear', 'surprise'],
  ['fear'],
  ['fear', 'anger'],
  ['fear', 'anger'],
  ['joy'],
  ['joy', 'anger']],
 [['joy'], ['anger', 'disgust'], ['surprise'], ['surprise', 'joy'], ['joy']],
 [['anger', 'fear'],
  ['fear', 'surprise'],
  ['anger', 'confusion'],
  ['joy', 'determination'],
  ['anger', 'defiance'],
  ['fear', 'uncertainty'],
  ['fear', 'caution'],
  ['anger', 'frustration'],
  ['anger',

In [183]:
grounds

["[['anger'], ['anger'], ['fear'], ['fear'], ['fear', 'sadness'], ['sadness'], ['anger'], ['surprise'], ['surprise'], ['fear', 'surprise']]",
 "[['fear'], ['anger'], ['surprise'], ['anger'], ['joy'], ['anger'], ['anger'], ['anger'], ['anger']]",
 "[['joy'], ['joy'], ['anger'], ['anger']]",
 "[['fear', 'surprise']]",
 "[['anger'], ['anger'], ['fear'], ['fear', 'surprise'], ['anger', 'joy'], ['anger']]",
 "[['sadness'], ['fear'], ['joy'], ['neutral'], ['surprise'], ['fear'], ['surprise'], ['joy'], ['surprise'], ['sadness', 'surprise']]",
 "[['surprise'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['joy']]",
 "[['fear'], ['fear'], ['anger'], ['fear'], ['anger', 'surprise'], ['fear'], ['fear'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['anger'], ['anger']]",
 "[['fear'], ['fear'], ['anger'], ['anger', 'fear', 'surprise'], ['fear', 'sadness'], ['fear', 'sadness'], ['anger'], ['anger'], ['surprise'], ['fear'], ['anger', 'surprise'], ['anger'], ['anger', 'sadness'], ['anger']]",

In [184]:
import ast

grounds = [ast.literal_eval(x) for x in grounds]

In [185]:
grounds

[[['anger'],
  ['anger'],
  ['fear'],
  ['fear'],
  ['fear', 'sadness'],
  ['sadness'],
  ['anger'],
  ['surprise'],
  ['surprise'],
  ['fear', 'surprise']],
 [['fear'],
  ['anger'],
  ['surprise'],
  ['anger'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger']],
 [['joy'], ['joy'], ['anger'], ['anger']],
 [['fear', 'surprise']],
 [['anger'],
  ['anger'],
  ['fear'],
  ['fear', 'surprise'],
  ['anger', 'joy'],
  ['anger']],
 [['sadness'],
  ['fear'],
  ['joy'],
  ['neutral'],
  ['surprise'],
  ['fear'],
  ['surprise'],
  ['joy'],
  ['surprise'],
  ['sadness', 'surprise']],
 [['surprise'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['joy']],
 [['fear'],
  ['fear'],
  ['anger'],
  ['fear'],
  ['anger', 'surprise'],
  ['fear'],
  ['fear'],
  ['fear', 'sadness'],
  ['surprise'],
  ['surprise'],
  ['anger'],
  ['anger']],
 [['fear'],
  ['fear'],
  ['anger'],
  ['anger', 'fear', 'surprise'],
  ['fear', 'sadness'],
  ['fear', 'sadness'],
  ['anger'],
  ['anger'],
  ['surpris

In [186]:
preds_l

[[['anger'],
  ['anger', 'disgust'],
  ['fear'],
  ['fear'],
  ['anger', 'disgust'],
  ['sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['surprise'],
  ['fear']],
 [['anger', 'disgust'],
  ['surprise', 'joy'],
  ['anger', 'fear'],
  ['anger', 'fear', 'surprise'],
  ['surprise', 'joy'],
  ['disgust'],
  ['neutral'],
  ['anger'],
  ['anger', 'joy']],
 [['anger'], ['anger', 'surprise'], ['anger', 'disgust'], ['joy', 'anger']],
 [['anger']],
 [['joy'],
  ['anger', 'joy'],
  ['anger'],
  ['anger', 'fear'],
  ['fear', 'surprise'],
  ['joy']],
 [['anger'],
  ['fear'],
  ['anger'],
  ['fear'],
  ['fear', 'surprise'],
  ['fear'],
  ['fear', 'anger'],
  ['fear', 'anger'],
  ['joy'],
  ['joy', 'anger']],
 [['joy'], ['anger', 'disgust'], ['surprise'], ['surprise', 'joy'], ['joy']],
 [['anger', 'fear'],
  ['fear', 'surprise'],
  ['anger', 'confusion'],
  ['joy', 'determination'],
  ['anger', 'defiance'],
  ['fear', 'uncertainty'],
  ['fear', 'caution'],
  ['anger', 'frustration'],
  ['anger',

In [187]:
bad_idx = []

for idx, (i,j) in enumerate(zip(grounds, preds_l)):
    if len(i) != len(j):
        print(idx, len(i), len(j))
        bad_idx.append(idx)

8 14 9
14 11 5
19 8 9
20 9 8
23 12 9
25 6 8
26 14 7
28 10 9
42 13 11
47 11 10
60 8 7
72 13 12
73 15 8
75 18 13
76 16 11
77 16 12
80 8 9
82 13 12
84 11 10
87 10 9
99 11 12
101 16 12
102 21 13
104 20 13
105 15 13
106 16 12
107 12 11
113 23 9
114 23 11
115 13 11
117 13 10
118 14 12
120 11 12
121 11 10
124 13 11
126 17 10
130 9 10
131 12 11
137 12 11
144 2 3
148 12 11
153 13 5
160 13 7
161 18 9
166 12 8
167 13 8
172 11 10
175 6 4
181 10 9
183 17 14
193 11 10
198 18 20
202 12 10
205 22 18
214 11 10
216 15 12
226 11 10
230 15 14
232 17 10
233 13 10
234 12 11
241 11 10
242 12 10
247 16 14
248 15 14
249 17 13
252 9 8
253 16 13
254 17 14
268 6 5
273 19 13
274 23 19
275 13 12
278 15 14
283 25 12
288 15 10
291 17 13
292 15 8
293 11 10
297 10 9
301 31 9
305 12 11
312 16 13
317 15 9
318 13 10
319 12 9
320 19 10
321 17 11
323 14 11
325 14 12
328 8 9
329 11 9
331 2 5
332 4 5
334 5 8
336 2 5
338 4 5
339 4 5
341 4 6
343 12 10
351 14 10
356 9 10
360 16 13
363 12 10
364 12 10
368 15 10
381 11 10
404 12 1

In [188]:
bad_idx.sort(reverse=True)

# Remove elements from 'grounds' at the specified indices
for idx in bad_idx:
    
    del grounds[idx]
    del preds_l[idx]

In [189]:
preds_l

[[['anger'],
  ['anger', 'disgust'],
  ['fear'],
  ['fear'],
  ['anger', 'disgust'],
  ['sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['surprise'],
  ['fear']],
 [['anger', 'disgust'],
  ['surprise', 'joy'],
  ['anger', 'fear'],
  ['anger', 'fear', 'surprise'],
  ['surprise', 'joy'],
  ['disgust'],
  ['neutral'],
  ['anger'],
  ['anger', 'joy']],
 [['anger'], ['anger', 'surprise'], ['anger', 'disgust'], ['joy', 'anger']],
 [['anger']],
 [['joy'],
  ['anger', 'joy'],
  ['anger'],
  ['anger', 'fear'],
  ['fear', 'surprise'],
  ['joy']],
 [['anger'],
  ['fear'],
  ['anger'],
  ['fear'],
  ['fear', 'surprise'],
  ['fear'],
  ['fear', 'anger'],
  ['fear', 'anger'],
  ['joy'],
  ['joy', 'anger']],
 [['joy'], ['anger', 'disgust'], ['surprise'], ['surprise', 'joy'], ['joy']],
 [['anger', 'fear'],
  ['fear', 'surprise'],
  ['anger', 'confusion'],
  ['joy', 'determination'],
  ['anger', 'defiance'],
  ['fear', 'uncertainty'],
  ['fear', 'caution'],
  ['anger', 'frustration'],
  ['anger',

In [190]:
grounds = [item for sublist in grounds for item in sublist]
predictions = [item for sublist in preds_l for item in sublist]

In [191]:
len(grounds), len(predictions)

(4336, 4336)

In [192]:
grounds

[['anger'],
 ['anger'],
 ['fear'],
 ['fear'],
 ['fear', 'sadness'],
 ['sadness'],
 ['anger'],
 ['surprise'],
 ['surprise'],
 ['fear', 'surprise'],
 ['fear'],
 ['anger'],
 ['surprise'],
 ['anger'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['anger'],
 ['anger'],
 ['joy'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['fear', 'surprise'],
 ['anger'],
 ['anger'],
 ['fear'],
 ['fear', 'surprise'],
 ['anger', 'joy'],
 ['anger'],
 ['sadness'],
 ['fear'],
 ['joy'],
 ['neutral'],
 ['surprise'],
 ['fear'],
 ['surprise'],
 ['joy'],
 ['surprise'],
 ['sadness', 'surprise'],
 ['surprise'],
 ['fear', 'sadness'],
 ['surprise'],
 ['surprise'],
 ['joy'],
 ['fear'],
 ['fear'],
 ['anger'],
 ['fear'],
 ['anger', 'surprise'],
 ['fear'],
 ['fear'],
 ['fear', 'sadness'],
 ['surprise'],
 ['surprise'],
 ['anger'],
 ['anger'],
 ['joy'],
 ['joy'],
 ['fear'],
 ['joy'],
 ['fear', 'sadness'],
 ['anger', 'disgust', 'surprise'],
 ['anger', 'disgust'],
 ['fear'],
 ['anger', 'disgust'],
 ['fear', 'surprise'],
 ['anger'],
 ['joy'],
 ['j

In [193]:
predictions

[['anger'],
 ['anger', 'disgust'],
 ['fear'],
 ['fear'],
 ['anger', 'disgust'],
 ['sadness'],
 ['fear', 'surprise'],
 ['surprise'],
 ['surprise'],
 ['fear'],
 ['anger', 'disgust'],
 ['surprise', 'joy'],
 ['anger', 'fear'],
 ['anger', 'fear', 'surprise'],
 ['surprise', 'joy'],
 ['disgust'],
 ['neutral'],
 ['anger'],
 ['anger', 'joy'],
 ['anger'],
 ['anger', 'surprise'],
 ['anger', 'disgust'],
 ['joy', 'anger'],
 ['anger'],
 ['joy'],
 ['anger', 'joy'],
 ['anger'],
 ['anger', 'fear'],
 ['fear', 'surprise'],
 ['joy'],
 ['anger'],
 ['fear'],
 ['anger'],
 ['fear'],
 ['fear', 'surprise'],
 ['fear'],
 ['fear', 'anger'],
 ['fear', 'anger'],
 ['joy'],
 ['joy', 'anger'],
 ['joy'],
 ['anger', 'disgust'],
 ['surprise'],
 ['surprise', 'joy'],
 ['joy'],
 ['anger', 'fear'],
 ['fear', 'surprise'],
 ['anger', 'confusion'],
 ['joy', 'determination'],
 ['anger', 'defiance'],
 ['fear', 'uncertainty'],
 ['fear', 'caution'],
 ['anger', 'frustration'],
 ['anger', 'determination'],
 ['anger', 'surprise'],
 ['a

In [194]:
mlb = MultiLabelBinarizer()

In [195]:
y_true_mhot = mlb.fit_transform(grounds)
y_pred_mhot = mlb.transform(predictions)

/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['admiration', 'aggression', 'agreement', 'anticipation', 'apologetic', 'appreciation', 'approval', 'assertion', 'calm', 'caution', 'comfort', 'concern', 'confidence', 'confusion', 'curiosity', 'defensiveness', 'defiance', 'determination', 'doubt', 'duty', 'encouragement', 'error', 'f', 'friendship', 'frustration', 'gratitude', 'guilt', 'hope', 'humor', 'investigation', 'love', 'memory', 'neutrality', 'pain', 'pleasure', 'politeness', 'pride', 'reassurance', 'relief', 'sarcasm', 'sarcastic', 'skepticism', 'sur', 'suspicion', 'trust', 'uncertainty', 'urgency'] will be ignored
  warnings.warn(


In [196]:
y_pred_mhot.shape

(4336, 7)

In [197]:
y_pred_mhot.shape

(4336, 7)

In [198]:
print(classification_report(y_true_mhot, y_pred_mhot, target_names=mlb.classes_, digits=3))

              precision    recall  f1-score   support

       anger      0.524     0.565     0.544      1534
     disgust      0.150     0.335     0.207       212
        fear      0.401     0.434     0.417      1118
         joy      0.476     0.450     0.463      1035
     neutral      0.104     0.349     0.160       238
     sadness      0.473     0.235     0.314       991
    surprise      0.402     0.361     0.380      1141

   micro avg      0.395     0.417     0.406      6269
   macro avg      0.361     0.390     0.355      6269
weighted avg      0.435     0.417     0.416      6269
 samples avg      0.401     0.426     0.389      6269



/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
